In [1]:
import numpy as np
import urllib
import os

# Download the dataset

In [2]:
# Get the webpage
url = 'https://landsat.usgs.gov/landsat-8-cloud-cover-assessment-validation-data'
response = urllib.request.urlopen(url)

# Find the data urls in the source
data_urls = []
for line in response:
    if 'https://landsat.usgs.gov/cloud-validation/cca' in str(line) and 'tar' in str(line):        
        data_urls.append(str(line)[47:124])

print("Products found:", np.size(data_urls))
print("Example url:   ", data_urls[0])        

Products found: 96
Example url:    https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz


In [3]:
# define backgrounds
backgrounds = ['Barren',
               'Forest',
               'GrassCrops',
               'Shrubland',
               'SnowIce',
               'Urban',
               'Water',
               'Wetlands']

In [4]:
# get test dataset
index = 0
for i in range(1):
    !wget {data_urls[i]} -P {'./tar/' + backgrounds[index] + '/'}

[0] Downloading 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz' ...
Saving './tar/Barren//LC80420082013220LGN00.tar.gz'
Just got 574303767 of 914473396 bytes (https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz)
HTTP response 200 OK [https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz]
Unexpected body length 574303767.
[0] Downloading 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz' ...
Saving './tar/Barren//LC80420082013220LGN00.tar.gz'
Just got 351607767 of 914473396 bytes (https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz)
HTTP response 200 OK [https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz]
Unexpected body length 351607767.
[0] Downloading 'https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz' ...
Saving './tar/Barren//LC80420082013220LGN00.tar.gz'
Just got 407863767 of 914473396 bytes 

In [13]:
import asyncio
import subprocess


index = -1
bulk_size = 12
remaining_dls = 12 # np.size(data_urls)


In [ ]:


loop = asyncio.get_event_loop()


async def download_tar_url(url, background):
    return await subprocess.Popen(f"wget {url} -P {'./tar/' + background + '/'}")
    #return await os.system(f"wget {url} -P {'./tar/' + background + '/'}")

async def download_bulk(bulk_size):
    for i in range(bulk_size):
        index+=1
        download_tar_url(data_urls[index+i], background=backgrounds[index%12])
    
async def download_all():
    while remaining_dls > 0:
        await download_bulk(bulk_size=bulk_size)
        remaining_dls -= bulk_size

asyncio.ensure_future(download_all())
loop.run_forever()

RuntimeError: This event loop is already running

Task exception was never retrieved
future: <Task finished name='Task-10' coro=<download_all() done, defined at /tmp/ipykernel_440/633997326.py:13> exception=UnboundLocalError("local variable 'remaining_dls' referenced before assignment")>
Traceback (most recent call last):
  File "/tmp/ipykernel_440/633997326.py", line 14, in download_all
    while remaining_dls > 0:
UnboundLocalError: local variable 'remaining_dls' referenced before assignment


In [ ]:
# Download all the products
index = -1
for i in range(0, np.size(data_urls)):
    if i%12 == 0:  # There are 12 scenes in every background category       
        index += 1     
    !wget {data_urls[i]+'.tar.gz'} -P {'./tar/' + backgrounds[index] + '/'}

http://www.gnu.org/software/parallel/man.html#example__download_10_images_for_each_of_the_past_30_days

Running 10000 jobs with 100 in parallel:
> seq 10000 | parallel -j100 wget https://www.example.com/page{}.html

In [ ]:
# Download all the products in background
index = -1
for i in range(0, np.size(data_urls)):
    if i%12 == 0:  # There are 12 scenes in every background category       
        index += 1
    !wget -b {data_urls[i]+'.tar.gz'} -P {'./tar/' + backgrounds[index] + '/'}

In [16]:
folders = [i for i in os.listdir("./tar/") if i not in [".gitignore", ".gitkeep"]]

!mkdir {"unzipped"}
for folder in folders:
    !mkdir {"./unzipped/" + folder}
    files = sorted(os.listdir("tar/" + folder))
    files = [f for f in files if '.tar.gz' in f] 
    for file in files:
        try: 
            print("----")
            print("Unzipping product:", file)
            !tar -xvzf {"./tar/" + folder + '/' + file} -C {'./unzipped/' + folder + '/'}
        except:
            print("----------------------------------------------------")
            print("UNZIP ERROR WITH PRODUCT:", file)
            pass

----
Unzipping product: LC80420082013220LGN00.tar.gz
BC/LC80420082013220LGN00/LC80420082013220LGN00_B10.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B11.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B1.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B2.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B3.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B4.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B5.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B6.TIF

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [15]:
# Convert the .img masks to .TIF masks
folders = sorted(os.listdir("./unzipped/"))
folders = [f for f in folders if '.' not in f]  # Filter out .gitignore
for folder in folders:
    products = sorted(os.listdir("./unzipped/" + folder + "/BC/"))
    for product in products:
        img_path  = "./unzipped/" + folder + "/BC/" + product + "/" + product + "_fixedmask.img"
        gtiff_path = "./unzipped/" + folder + "/BC/" + product + "/" + product + "_fixedmask.TIF"
        !gdal_translate -of GTiff {img_path + " " + gtiff_path}

Input file size is 8911, 8941
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 8671, 8721
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 9101, 9071
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7611, 7751
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7871, 7991
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7781, 7901
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 8931, 8961
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7551, 7721
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7551, 7741
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7571, 7391
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7511, 7321
0...10...20...30...40...50...60...70...80...90...100 - done.

In [12]:
# Move to raw data folder
project_path = "/home/jhj/phd/GitProjects/SentinelSemanticSegmentation/"
!cp -R ./unzipped/* {project_path + 'data/raw/'}

In [ ]:
# Download SPARCS dataset
!wget https://landsat.usgs.gov/cloud-validation/sparcs/l8cloudmasks.zip -P ./zip/